In [7]:
import pandas as pd
import os
import re
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [8]:
def parse_logs(filename):
    with open(filename) as f:
        content = f.readlines()        
        
    content = [x.strip() for x in content] 
    results = []
    
    for line in content:    
        try:
            data = {}
            data['passed'] = bool(re.search("passed", line, flags=re.IGNORECASE))
            data['name'] = re.search("isis3_[A-z]+|[A-z]+_app_[A-z]+", line, flags=re.IGNORECASE).group(0)
            data['test_id'] = int(re.search("#[0-9]+", line, flags=re.IGNORECASE).group(0)[1:])
            data['time'] = float(re.search("[0-9]+\.[0-9]+", line, flags=re.IGNORECASE).group(0))
            results.append(data)
        except Exception as e:
            pass
    
    return pd.DataFrame(results)

In [71]:
def parse_make(filename):
    with open(filename) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
    results = []
    # Set initial data and target to none since the re.match may not find anything
    data = {'Target':None}
    data_list = []
    target = None
    
    for line in content:
        try:
            # If nothing is found, the .group(0) call will cause an error and
            # move to the exception clause
            target = re.match("Scanning dependencies of target [A-z or 0-9 or _]+|[A-z or 0-9 or _]+.so", line,
                              flags=re.IGNORECASE).group(0)[32:]
            # If there is a new target
            if data['Target'] != target:
                # And the current target is not None
                if data['Target']:
                    # Add the current data to the result set
                    results.append(data)
                # No matter what the target is clear the data_list and update the next data point with
                # the new target, empty data_list, and set built to true
                data_list = []
                data = {'Target':target, 'data_list':data_list, 'built':True}
                
        except Exception as e:
            # If there is a dependency we are building
            if target:
                # Check to see if the line has some kind of error or warning
                error_line = re.search("fatal error:|error:|warning:", line, flags=re.IGNORECASE)
                # If there is an error it "probably" failed to build (Hasn't been check with all
                # of the various errors we can get)
                if error_line:
                    data['built'] = False
                data_list.append(line)
            pass
    results.append(data)
    return pd.DataFrame(results)